In [1]:
%%bash
docker run \
  --rm \
  --name omnisci_test \
  --ipc=host \
  -d --runtime=nvidia \
  -v $(readlink -f ..)/data:/data \
  -v $(readlink -f ..)/scripts:/src \
  -p 6273-6280:6273-6280 \
  omnisci/core-os-cuda:v5.2.1

ff8887943e2e1e62f4eac3fa0856053f1310a7bf8c3f7a961d0547fe0ade39b3


In [2]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "cat /src/load_dataset.sql | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Result
Loaded: 1067371 recs, Rejected: 0 recs in 0.349000 secs
1 rows returned.
Execution time: 438 ms, Total time: 438 ms
Result
Loaded: 5305 recs, Rejected: 0 recs in 0.027000 secs
1 rows returned.
Execution time: 33 ms, Total time: 33 ms
User admin disconnected from database omnisci


In [3]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "echo 'SELECT * FROM online_retail LIMIT 10;' | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Invoice|StockCode|Description|Quantity|InvoiceDate|Price|CustomerID|Country
490209|22200|FRYING PAN PINK POLKADOT|72|2009-12-04 12:06:00|4.25|17389|United Kingdom
490209|85099B|JUMBO BAG RED WHITE SPOTTY|100|2009-12-04 12:06:00|1.65|17389|United Kingdom
490209|21916|SET 12 RETRO WHITE CHALK STICKS|96|2009-12-04 12:06:00|0.42|17389|United Kingdom
491888|21171|BATHROOM METAL SIGN|4|2009-12-14 15:16:00|1.45|15867|United Kingdom
490212|21231|SWEETHEART CERAMIC TRINKET BOX|8|2009-12-04 12:07:00|1.25|16883|United Kingdom
491665|82552|WASHROOM METAL SIGN|3|2009-12-13 11:32:00|1.25|14685|United Kingdom
491888|85047|WHITE BEADED GARLAND STRING 20LIGHT|24|2009-12-14 15:16:00|5.55|15867|United Kingdom
490212|22296|HEART IVORY TRELLIS LARGE|3|2009-12-04 12:07:00|1.65|16883|United Kingdom
490803|22197|POPCORN HOLDER , SMALL |24|2009-12-08 12:04:00|0.85|17067|United Kingdom
491888|72807A|SET/3 ROSE CANDLE IN JEWELLED BOX|9|2009-12-14 15:16:00|4.25|15867|Unite

In [27]:
from pymapd import connect
from time import time
import pandas as pd
import numpy as np
import json

In [28]:
with open('test_result.json') as json_file:
    test_result = json.load(json_file)

In [6]:
con = connect(user="admin", password="HyperInteractive", host="localhost", dbname="omnisci")

In [31]:
test_result["timings"]["omnisci"] = {}
query_index = 1

for query in test_result["queries"]:
    label = f"Q{query_index}"
    print(f"Executing {label}")
    test_result["timings"]["omnisci"][label] = []
    for i in range(0, 10):
        start = time()
        pd.read_sql_query(query, con=con)  # <-un po' più veloce ma usa ipc (arrow)
        # df = con.select_ipc_gpu(query)  # <-analogo
        # result = list(con.execute(query))  #NO IPC
        end = time()

        total = end-start
        test_result["timings"]["omnisci"][label].append(total)
    query_index += 1

with open('test_result.json', 'w') as outfile:
    json.dump(test_result, outfile)

Executing Q1
Executing Q2
Executing Q3
Executing Q4
Executing Q5
Executing Q6
Executing Q7


In [18]:
# TEST ON JOINS

query = "SELECT Invoice, stock_codes.Price FROM online_retail JOIN stock_codes ON online_retail.StockCode = stock_codes.StockCode"

start = time()
pd.read_sql_query(query, con=con)
end = time()

total = end-start
print(total)

1.3599514961242676


In [19]:
# TEST ON JOINS

query = "SELECT Invoice, Price FROM online_retail"

start = time()
pd.read_sql_query(query, con=con)
end = time()

total = end-start
print(total)

1.328507900238037


In [57]:
%%bash
docker stop omnisci_test

omnisci_test
